In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/FaceMask/face_mask_test_corrette.txt /content/
!cp /content/drive/My\ Drive/FaceMask/face_mask_train_corrette.txt /content/
!unzip -q /content/drive/My\ Drive/FaceMask/FaceMaskDataset.zip

replace train/test_00000127.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
!git clone https://github.com/qqwweee/keras-yolo3.git

import os
for i in os.listdir("keras-yolo3/"):
  os.system("mv keras-yolo3/"+i+" /content/")

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.07 KiB | 585.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


In [0]:
!pip install keras==2.1.5

%tensorflow_version 1.x
import tensorflow as tf
import sys
import argparse
from yolo import YOLO
from PIL import Image
import os
from yolo3.model import *
from yolo3.utils import *
tf.version.VERSION

'1.15.2'

In [0]:
from IPython.display import display

def detecting_image(detector, image):
    bb = []
    classi = []
    if detector.model_image_size != (None, None):
        assert detector.model_image_size[0] % 32 == 0, 'Multiples of 32 required'
        assert detector.model_image_size[1] % 32 == 0, 'Multiples of 32 required'
        boxed_image = letterbox_image(image, tuple(reversed(detector.model_image_size)))
    else:
        new_image_size = (image.width - (image.width % 32),
                          image.height - (image.height % 32))
        boxed_image = letterbox_image(image, new_image_size)
    image_data = np.array(boxed_image, dtype='float32')

    # print(image_data.shape)
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

    out_boxes, out_scores, out_classes = detector.sess.run(
        [detector.boxes, detector.scores, detector.classes],
        feed_dict={
            detector.yolo_model.input: image_data,
            detector.input_image_shape: [image.size[1], image.size[0]],
            K.learning_phase(): 0
        })

    # print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = detector.class_names[c]
        box = out_boxes[i]
        score = out_scores[i]

        top, left, bottom, right = box

        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        # print(label, (left, top), (right, bottom))

        bb.append([left, top, right, bottom])
        if predicted_class == "face":
            classi.append(0)
        else:
            classi.append(1)

    bb_final = []
    classes_final = []
    for index in range(0,len(classi)):
        if classi[index] == 1:
            bb_final.append(bb[index])
            classes_final.append(classi[index])

    bb_0 = []
    for index in range(0, len(classi)):
        if classi[index] == 0:
            flag = True
            for mask in bb_final:
                iou = bb_intersection_over_union(mask,bb[index])
                if iou>0.5:
                    flag = False
                    break
            if flag:
                bb_0.append(bb[index])
    for i in bb_0:
        bb_final.append(i)
        classes_final.append(0)

    return bb_final, classes_final

In [0]:
f = open("classes.txt","w")
f.write("face\n")
f.write("face_mask")
f.close()

In [0]:
%tensorflow_version 1.x
import cv2
import os
from PIL import Image, ImageFont, ImageDraw
import numpy as np
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
from IPython.display import display

def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


In [0]:
!git clone https://github.com/swdev1202/keras-yolo3-facedetection.git


fatal: destination path 'keras-yolo3-facedetection' already exists and is not an empty directory.


In [0]:
def compute_performance(predinput, trueinput, pred_classes, true_classes):
  pred = []
  true = []
  for x in range(0,len(predinput)):
    pred.append({'p':predinput[x], 'matched':False, 'class':pred_classes[x]})
  for x in range(0,len(trueinput)):
    true.append({'p':trueinput[x], 'matched':False, 'class':true_classes[x]})

  ce = 0
  tp = 0
  fp = 0
  fn = 0

  for p in pred:
    for t in true:
      if not t['matched'] and not p['matched']:
        try:
          iou = bb_intersection_over_union(p['p'], t['p'])
                    

          if iou > 0.5:
            t['matched'] = True
            p['matched'] = True
            if p['class'] == t['class']:
              tp += 1
            else:
              ce += 1
        except:
          for t in true:
            if not t['matched']:
              fn += 1
          print("Error computing performance")
                    
                   
    if not p['matched']:
      fp +=1
  for t in true:
    if not t['matched']:
      fn += 1
  
  return (tp,ce,fp,fn)

In [0]:
#############################

  #  yolo face

##############################

from IPython.display import display
from random import randint
from PIL import Image, ImageDraw
import progressbar


model_path = "/content/drive/My Drive/FaceMask/YoloFaceDataAug/ep068-loss9.167-val_loss9.443.h5"
txt = open("face_mask_test_corrette.txt","r")
label = txt.readlines()
label = label[:len(label)-1]
txt.close()

tpTot = 0
ceTot = 0
fpTot = 0
fnTot = 0
totBB = 0

arg = {
      "model_path": model_path,
      "anchors_path": '/content/keras-yolo3-facedetection/model_data/wider_anchors.txt',  #YOLOFACE
      "classes_path": 'classes.txt',
      "score" : 0.3,
      "iou" : 0.45,
      "model_image_size" : (608,608),  #YOLOFACE
      "gpu_num" : 1,
  }
detector = YOLO(**arg)

bar = progressbar.ProgressBar(max_value=len(label))
cont = 0

for i in label:
  s = i.split(" ")
  true = []
  true_classes = []
  image = Image.open(s[0])
  for j in range(1,len(s)):
    punti = s[j].split(",")
    try:
      true.append([int(punti[0]),int(punti[1]),int(punti[2]),int(punti[3]) ])
      true_classes.append(int(punti[4]))
    except:
      print(i)
      for punto in punti:
        print(punto)
  
  predicted,pred_classes = detecting_image(detector, image)
  tp,ce,fp,fn = compute_performance(predicted, true, pred_classes, true_classes)

  tpTot += tp
  ceTot += ce
  fpTot += fp
  fnTot += fn

  bar.update(cont)
  cont = cont+1
  totBB +=len(true)
  # print("Confronto immagine numero : "+str(cont)+" BB presenti:"+str(len(true))+" BB trovati: "+str(len(predicted)))
  # print(pred_classes)


print("BB true totali: "+str(totBB))
print("True positive: "+str(tpTot))
print("Classification error: "+str(ceTot))
print("False positive: "+str(fpTot))
print("False negative: "+str(fnTot))

R = tpTot/totBB
P = tpTot/(tpTot+fpTot)
A = tpTot/(tpTot+ceTot)

print("Recall: "+str(R))
print("Precision: "+str(P))
print("Accuracy: "+str(A))
  


/content/drive/My Drive/FaceMask/YoloFaceDataAug/ep068-loss9.167-val_loss9.443.h5 model, anchors, and classes loaded.


 99% (1837 of 1838) |################### | Elapsed Time: 0:05:44 ETA:   0:00:00

BB true totali: 3061
True positive: 2829
Classification error: 93
False positive: 118
False negative: 139
Recall: 0.9242077752368507
Precision: 0.9599592806243638
Accuracy: 0.9681724845995893


In [0]:

####################################

    # yolo v3 doppio training

####################################

from IPython.display import display
from random import randint
from PIL import Image, ImageDraw
import progressbar


model_path = "/content/drive/My Drive/FaceMask/DarknetYoloNoDataAug/trainingcompleto/ep080-loss8.454-val_loss8.272.h5"
txt = open("face_mask_test_corrette.txt","r")
label = txt.readlines()
label = label[:len(label)-1]
txt.close()

tpTot = 0
ceTot = 0
fpTot = 0
fnTot = 0
totBB = 0

arg = {
      "model_path": model_path,
      "anchors_path": 'model_data/yolo_anchors.txt', #YOLOV3
      "classes_path": 'classes.txt',
      "score" : 0.3,
      "iou" : 0.45,
      "model_image_size" : (416, 416), #YOLOV3
      "gpu_num" : 1,
  }
detector = YOLO(**arg)

bar = progressbar.ProgressBar(max_value=len(label))
cont = 0

for i in label:
  s = i.split(" ")
  true = []
  true_classes = []
  image = Image.open(s[0])
  for j in range(1,len(s)):
    punti = s[j].split(",")
    try:
      true.append([int(punti[0]),int(punti[1]),int(punti[2]),int(punti[3]) ])
      true_classes.append(int(punti[4]))
    except:
      print(i)
      for punto in punti:
        print(punto)
  
  predicted,pred_classes = detecting_image(detector, image)
  tp,ce,fp,fn = compute_performance(predicted, true, pred_classes, true_classes)

  tpTot += tp
  ceTot += ce
  fpTot += fp
  fnTot += fn

  bar.update(cont)
  cont = cont+1
  totBB +=len(true)
  # print("Confronto immagine numero : "+str(cont)+" BB presenti:"+str(len(true))+" BB trovati: "+str(len(predicted)))
  # print(pred_classes)


print("BB true totali: "+str(totBB))
print("True positive: "+str(tpTot))
print("Classification error: "+str(ceTot))
print("False positive: "+str(fpTot))
print("False negative: "+str(fnTot))

R = tpTot/totBB
P = tpTot/(tpTot+fpTot)
A = tpTot/(tpTot+ceTot)

print("Recall: "+str(R))
print("Precision: "+str(P))
print("Accuracy: "+str(A))


/content/drive/My Drive/FaceMask/DarknetYoloNoDataAug/trainingcompleto/ep080-loss8.454-val_loss8.272.h5 model, anchors, and classes loaded.


 99% (1837 of 1838) |################### | Elapsed Time: 0:03:40 ETA:   0:00:00

BB true totali: 3061
True positive: 2779
Classification error: 99
False positive: 108
False negative: 183
Recall: 0.9078732440378962
Precision: 0.9625909248354694
Accuracy: 0.9656011118832523


In [0]:

##################

    # yolo v3

##################

from IPython.display import display
from random import randint
from PIL import Image, ImageDraw
import progressbar


model_path = "/content/drive/My Drive/FaceMask/DarknetYoloDataAug/ep054-loss8.067-val_loss8.021.h5"
txt = open("face_mask_test_corrette.txt","r")
label = txt.readlines()
label = label[:len(label)-1]
txt.close()

tpTot = 0
ceTot = 0
fpTot = 0
fnTot = 0
totBB = 0

arg = {
      "model_path": model_path,
      "anchors_path": 'model_data/yolo_anchors.txt', #YOLOV3
      "classes_path": 'classes.txt',
      "score" : 0.3,
      "iou" : 0.45,
      "model_image_size" : (416, 416), #YOLOV3
      "gpu_num" : 1,
  }
detector = YOLO(**arg)

bar = progressbar.ProgressBar(max_value=len(label))
cont = 0

for i in label:
  s = i.split(" ")
  true = []
  true_classes = []
  image = Image.open(s[0])
  for j in range(1,len(s)):
    punti = s[j].split(",")
    try:
      true.append([int(punti[0]),int(punti[1]),int(punti[2]),int(punti[3]) ])
      true_classes.append(int(punti[4]))
    except:
      print(i)
      for punto in punti:
        print(punto)
  
  predicted,pred_classes = detecting_image(detector, image)
  tp,ce,fp,fn = compute_performance(predicted, true, pred_classes, true_classes)

  tpTot += tp
  ceTot += ce
  fpTot += fp
  fnTot += fn

  bar.update(cont)
  cont = cont+1
  totBB +=len(true)
  # print("Confronto immagine numero : "+str(cont)+" BB presenti:"+str(len(true))+" BB trovati: "+str(len(predicted)))
  # print(pred_classes)


print("BB true totali: "+str(totBB))
print("True positive: "+str(tpTot))
print("Classification error: "+str(ceTot))
print("False positive: "+str(fpTot))
print("False negative: "+str(fnTot))

R = tpTot/totBB
P = tpTot/(tpTot+fpTot)
A = tpTot/(tpTot+ceTot)

print("Recall: "+str(R))
print("Precision: "+str(P))
print("Accuracy: "+str(A))


/content/drive/My Drive/FaceMask/DarknetYoloDataAug/ep054-loss8.067-val_loss8.021.h5 model, anchors, and classes loaded.


 99% (1837 of 1838) |################### | Elapsed Time: 0:03:40 ETA:   0:00:00

BB true totali: 3061
True positive: 2802
Classification error: 92
False positive: 110
False negative: 167
Recall: 0.9153871283894153
Precision: 0.9622252747252747
Accuracy: 0.9682100898410505


In [0]:

#####################

  # tiny yolo v3

#####################

from IPython.display import display
from random import randint
from PIL import Image, ImageDraw
import progressbar

model_path = "/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep053-loss10.014-val_loss9.798.h5"
txt = open("face_mask_test_corrette.txt","r")
label = txt.readlines()
label = label[:len(label)-1]
txt.close()

tpTot = 0
ceTot = 0
fpTot = 0
fnTot = 0
totBB = 0

arg = {
      "model_path": model_path,
      "anchors_path": '/content/model_data/tiny_yolo_anchors.txt', #YOLOV3
      "classes_path": 'classes.txt',
      "score" : 0.3,
      "iou" : 0.45,
      "model_image_size" : (416, 416), #YOLOV3
      "gpu_num" : 1,
  }
detector = YOLO(**arg)

bar = progressbar.ProgressBar(max_value=len(label))
cont = 0

for i in label:
  s = i.split(" ")
  true = []
  true_classes = []
  image = Image.open(s[0])
  for j in range(1,len(s)):
    punti = s[j].split(",")
    try:
      true.append([int(punti[0]),int(punti[1]),int(punti[2]),int(punti[3]) ])
      true_classes.append(int(punti[4]))
    except:
      print(i)
      for punto in punti:
        print(punto)
  
  predicted,pred_classes = detecting_image(detector, image)
  tp,ce,fp,fn = compute_performance(predicted, true, pred_classes, true_classes)

  tpTot += tp
  ceTot += ce
  fpTot += fp
  fnTot += fn

  bar.update(cont)
  cont = cont+1
  totBB +=len(true)
  # print("Confronto immagine numero : "+str(cont)+" BB presenti:"+str(len(true))+" BB trovati: "+str(len(predicted)))
  # print(pred_classes)


print("BB true totali: "+str(totBB))
print("True positive: "+str(tpTot))
print("Classification error: "+str(ceTot))
print("False positive: "+str(fpTot))
print("False negative: "+str(fnTot))

R = tpTot/totBB
P = tpTot/(tpTot+fpTot)
A = tpTot/(tpTot+ceTot)

print("Recall: "+str(R))
print("Precision: "+str(P))
print("Accuracy: "+str(A))
  











/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep053-loss10.014-val_loss9.798.h5 model, anchors, and classes loaded.


 99% (1835 of 1838) |################### | Elapsed Time: 0:01:26 ETA:   0:00:00

BB true totali: 3061
True positive: 2516
Classification error: 90
False positive: 128
False negative: 455
Recall: 0.821953609931395
Precision: 0.9515885022692889
Accuracy: 0.965464313123561


In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  7 16:05:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    58W / 149W |   6894MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:

# #####################

#   # tiny yolo v3

# #####################

# from IPython.display import display
# from random import randint
# from PIL import Image, ImageDraw
# import progressbar


# def tiny_yolo_test(score,iou):

#   model_path = "/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep053-loss10.014-val_loss9.798.h5"
#   txt = open("face_mask_test_corrette.txt","r")
#   label = txt.readlines()
#   label = label[:len(label)-1]
#   txt.close()

#   tpTot = 0
#   ceTot = 0
#   fpTot = 0
#   fnTot = 0
#   totBB = 0

#   arg = {
#         "model_path": model_path,
#         "anchors_path": '/content/model_data/tiny_yolo_anchors.txt', #YOLOV3
#         "classes_path": 'classes.txt',
#         "score" : score,
#         "iou" : iou,
#         "model_image_size" : (416, 416), #YOLOV3
#         "gpu_num" : 1,
#     }
#   detector = YOLO(**arg)

#   bar = progressbar.ProgressBar(max_value=len(label))
#   cont = 0

#   for i in label:
#     s = i.split(" ")
#     true = []
#     true_classes = []
#     image = Image.open(s[0])
#     for j in range(1,len(s)):
#       punti = s[j].split(",")
#       try:
#         true.append([int(punti[0]),int(punti[1]),int(punti[2]),int(punti[3]) ])
#         true_classes.append(int(punti[4]))
#       except:
#         print(i)
#         for punto in punti:
#           print(punto)
    
#     predicted,pred_classes = detecting_image(detector, image)
#     tp,ce,fp,fn = compute_performance(predicted, true, pred_classes, true_classes)

#     tpTot += tp
#     ceTot += ce
#     fpTot += fp
#     fnTot += fn

#     bar.update(cont)
#     cont = cont+1
#     totBB +=len(true)
#     # print("Confronto immagine numero : "+str(cont)+" BB presenti:"+str(len(true))+" BB trovati: "+str(len(predicted)))
#     # print(pred_classes)


#   # print("BB true totali: "+str(totBB))
#   # print("True positive: "+str(tpTot))
#   # print("Classification error: "+str(ceTot))
#   # print("False positive: "+str(fpTot))
#   # print("False negative: "+str(fnTot))

#   R = tpTot/totBB
#   P = tpTot/(tpTot+fpTot)
#   A = tpTot/(tpTot+ceTot)

#   print("Performance con score pari a "+str(score)+" e IOU pari a "+str(iou))
#   print("Recall: "+str(R))
#   print("Precision: "+str(P))
#   print("Accuracy: "+str(A))



# score = 0.3
# iou = 0.2
# for i in range(0,6):
#   tiny_yolo_test(score,iou+i*0.05)

# score = 0.2
# iou = 0.45
# for i in range(0,4):
#   tiny_yolo_test(score+i*0.05,iou)

    














/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep053-loss10.014-val_loss9.798.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


 87% (1607 of 1838) |#################   | Elapsed Time: 0:04:54 ETA:   0:00:42

val/test_00000306.jpg 





 99% (1837 of 1838) |################### | Elapsed Time: 0:05:39 ETA:   0:00:00

Performance con score pari a 0.3 e IOU pari a 0.2
Recall: 0.7750898986596927
Precision: 0.9367838798893718
Accuracy: 0.9101727447216891
/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep053-loss10.014-val_loss9.798.h5 model, anchors, and classes loaded.


...

Performance con score pari a 0.30000000000000004 e IOU pari a 0.45
Recall: 0.7750898986596927
Precision: 0.9330972058244785
Accuracy: 0.9101727447216891
/content/drive/My Drive/FaceMask/TinyYoloDataAug/ep053-loss10.014-val_loss9.798.h5 model, anchors, and classes loaded.


 87% (1607 of 1838) |#################   | Elapsed Time: 0:04:54 ETA:   0:00:42

val/test_00000306.jpg 





 99% (1837 of 1838) |################### | Elapsed Time: 0:05:40 ETA:   0:00:00

Performance con score pari a 0.35000000000000003 e IOU pari a 0.45
Recall: 0.7672441974501472
Precision: 0.9475171578522407
Accuracy: 0.916796875
